In [1]:
import cv2
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import random
from PIL import Image
from core.utils import IoU, assemble_data, convert_to_square
from core.models import creat_mtcnn_net
from core.utils import generate_bounding_box, nms
from core.detect import detect_pnet

In [2]:
anno_file = '/Users/shaotianyuan/DATA/WIDER/wider_face_split/wider_face_train_bbx_gt.txt'
im_dir = '/Users/shaotianyuan/DATA/WIDER/WIDER_train/images'

pos_save_dir = './data_set/train/24/positive'
part_save_dir = './data_set/train/24/part'
neg_save_dir = './data_set/train/24/negative'

if not os.path.exists(pos_save_dir):
    os.makedirs(pos_save_dir)
if not os.path.exists(part_save_dir):
    os.mkdir(part_save_dir)
if not os.path.exists(neg_save_dir):
    os.mkdir(neg_save_dir)
    
f1 = open(os.path.join('./anno_store', 'pos_24.txt'), 'w')
f2 = open(os.path.join('./anno_store', 'neg_24.txt'), 'w')
f3 = open(os.path.join('./anno_store', 'part_24.txt'), 'w')


p_idx = 0
n_idx = 0
d_idx = 0
image_idx = 0
box_idx = 0

pnet_path = '/Users/shaotianyuan/PycharmProjects/face_detection/mtcnn/model_store/pnet.pkl'

In [3]:
with open(anno_file) as f:
    annotations = f.readlines()

n = -1
lines = []
for annotation in annotations:
    annotation = annotation.strip().split(' ')
    if annotation[0][-3:] == 'jpg':
        lines.append(annotation)
        n += 1
    else:
        annotation = list(map(int, annotation))
        lines[n].extend(annotation[:4])
print(len(lines))
print(lines[:2])

12880
[['0--Parade/0_Parade_marchingband_1_849.jpg', 1, 449, 330, 122, 149], ['0--Parade/0_Parade_Parade_0_904.jpg', 1, 361, 98, 263, 339]]


In [4]:
for annotation in lines[:200]:
    im_path = os.path.join(im_dir, annotation[0])
    bbox = list(map(float, annotation[2:]))
    boxes = np.array(bbox, dtype=np.int32).reshape(-1, 4)
    boxes[:, 2] += boxes[:, 0] - 1
    boxes[:, 3] += boxes[:, 1] - 1
    
    img = np.array(Image.open(im_path).convert('RGB'))
    gts = boxes
    _, dets = detect_pnet(img, pnet_path)
    
    image_idx += 1
    if image_idx % 10 == 0:
        print(image_idx, "images done")
    
    if dets is None or dets.shape[0] == 0:
        continue
  
    dets = convert_to_square(dets)
    dets[:, :4] = np.round(dets[:, :4])
    
    neg_num = 0
    
    for box in dets:
        x_left, y_top, x_right, y_bottom, _ = box.astype(int)
        width = x_right - x_left + 1
        height = y_bottom - y_top + 1
        
        if width < 20 or x_left < 0 or y_top < 0 or x_right > img.shape[1] - 1 or y_bottom > img.shape[0] - 1:
            continue
            
        Iou = IoU(box, gts)
        cropped_im = img[y_top:y_bottom + 1, x_left:x_right + 1, :]
        resized_im = cv2.resize(cropped_im, (24, 24), interpolation=cv2.INTER_LINEAR)
        
        if np.max(Iou) < 0.3 and neg_num < 60:
            # save the examples
            save_file = os.path.join(neg_save_dir, "%s.jpg" % n_idx)
            # print(save_file)
            f2.write(save_file + ' 0 %.2f %.2f %.2f %.2f\n' % (0, 0, 0, 0))
            img_save = Image.fromarray(resized_im)
            img_save.save(save_file)
            n_idx += 1
            neg_num += 1
        else:
            # find gt_box with the highest iou
            idx = np.argmax(Iou)
            assigned_gt = gts[idx]
            x1, y1, x2, y2 = assigned_gt

            # compute bbox reg label
            offset_x1 = (x1 - x_left) / float(width)
            offset_y1 = (y1 - y_top) / float(height)
            offset_x2 = (x2 - x_right) / float(width)
            offset_y2 = (y2 - y_bottom) / float(height)

            # save positive and part-face images and write labels
            if np.max(Iou) >= 0.65:
                save_file = os.path.join(pos_save_dir, "%s.jpg" % p_idx)
                f1.write(save_file + ' 1 %.2f %.2f %.2f %.2f\n' % (
                    offset_x1, offset_y1, offset_x2, offset_y2))
                img_save = Image.fromarray(resized_im)
                img_save.save(save_file)
                p_idx += 1

            elif np.max(Iou) >= 0.4:
                save_file = os.path.join(part_save_dir, "%s.jpg" % d_idx)
                f3.write(save_file + ' -1 %.2f %.2f %.2f %.2f\n' % (
                    offset_x1, offset_y1, offset_x2, offset_y2))
                img_save = Image.fromarray(resized_im)
                img_save.save(save_file)
                d_idx += 1
f1.close()
f2.close()
f3.close()
print()
print("%s images done, pos: %s part: %s neg: %s" % (image_idx, p_idx, d_idx, n_idx))

10 images done
20 images done
30 images done
40 images done
50 images done
60 images done
70 images done
80 images done
90 images done
100 images done
110 images done
120 images done
130 images done
140 images done
150 images done
160 images done
170 images done
180 images done
190 images done
200 images done

200 images done, pos: 2342 part: 14680 neg: 11954


In [5]:
anno_list = ['./anno_store/neg_24.txt', './anno_store/part_24.txt', './anno_store/pos_24.txt']
imglist_filename = './anno_store/imglist_anno_24.txt'
chose_count = assemble_data(imglist_filename, anno_list)
chose_count

./anno_store/neg_24.txt
./anno_store/part_24.txt
./anno_store/pos_24.txt


28976